# GPU 연동 확인

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15281247317931586966,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4409662798050905327
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3048629863
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 411006660113267433
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:02:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14068690066283100003
 physical_device_desc: "device: XLA_GPU device"]

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
gpus = tf.config.experimental.list_physical_devices('XLA_GPU')
print(gpus)
tf.config.experimental.set_visible_devices(gpus[0], 'XLA_GPU')

tf.test.is_gpu_available()

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]


True

# Naver 영화 리뷰 감정분석
데이터 셋은 https://github.com/e9t/nsmc 의 자료를 사용

In [4]:
import urllib.request
import pandas as pd

## data 가져오기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="total_data.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="train_data.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="test_data.txt")

total_data = pd.read_table("total_data.txt")            # read_table시 자동으로 strip() 적용
train_data = pd.read_table("train_data.txt")
test_data = pd.read_table("test_data.txt")

print("총 데이터 개수: ", len(total_data))
print("훈련 데이터 개수:", len(train_data))
print("검증 데이터 개수:", len(test_data))

총 데이터 개수:  200000
훈련 데이터 개수: 150000
검증 데이터 개수: 50000


## 데이터 __확인__

In [5]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 데이터 전처리 진행
데이터의 결측값, 이상값, 빈값, 중복 등을 처리하여 정제된 데이터 셋으로 구성

In [6]:
import pandas as pd

# 1. 정규표현식 적용
train_data["document"] = train_data['document'].str.replace("[^0-9^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True).str.replace("[ㄱ-ㅎㅏ-ㅣ]", "", regex=True).str.replace("^", "", regex=False)
test_data["document"] = test_data['document'].str.replace("[^0-9^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True).str.replace("[ㄱ-ㅎㅏ-ㅣ]", "", regex=True).str.replace("^", "", regex=False)

In [7]:
# 맞춤법 및 띄어쓰기 교정
from hanspell import spell_checker
from tqdm import tqdm
import os

correction_document = []
for i in tqdm(range(train_data.shape[0])):
    try:
        cor_doc = spell_checker.check(train_data['document'][i])
        correction_document.append(cor_doc.checked)
    except:
        correction_document.append("")
    # 106분 소요

train_data['document'] = pd.DataFrame(correction_document)

train_data

100%|████████████████████████████████████████████████████████████████████████| 150000/150000 [1:47:55<00:00, 23.17it/s]


,id,document,label
0,9976970,아 더빙 진짜 짜증 나네요 목소리,0
1,3819312,흠포스터보고 초등학생영화 줄 오버 연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이먼 페그의 익살스러운 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔 죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1


In [8]:
# 맞춤법 교정 - 계속
from tqdm import tqdm        # p-var

from hanspell import spell_checker

correction_document = []
for i in tqdm(range(test_data.shape[0])):
    try:
        cor_doc = spell_checker.check(test_data['document'][i])
        correction_document.append(cor_doc.checked)
    except:
        correction_document.append("")
    # 35분 소요

test_data['document'] = pd.DataFrame(correction_document)

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [1:41:32<00:00,  8.21it/s]


In [9]:
# Id 별 중복되는 document 확인 및 삭제
# 다만, id는 다르지만 중복된 리뷰라면 개개인의 의견이니 제거 하지 않는다.
train_data["id"].is_unique        # is_unique ; res => boolean True - 중복 없음 / False - 중복 있음  # True
test_data['id'].is_unique         # True

True

In [10]:
import os.path
import pickle
## train_data 진행
file = 'cor_train_data.txt'

if os.path.isfile(file):
    print(f'{file} 파일이 존재합니다. 불러옵니다.')
    with open('cor_train_data.txt', 'rb') as f:
        cor_train_data = pickle.load(f)
        train_data['document'] = pd.DataFrame(cor_train_data)
        print("파일을 불러왔습니다.")
else:
    with open("cor_train_data.txt", 'wb') as F:
        pickle.dump(correction_document, F, pickle.HIGHEST_PROTOCOL)
        print("파일을 생성하였습니다.")
        
        
## test_data 진행
file = 'cor_test_data.txt'

if os.path.isfile(file):
    print(f'{file} 파일이 존재합니다. 불러옵니다.')
    with open('cor_test_data.txt', 'rb') as f:
        cor_test_data = pickle.load(f)
        test_data['document'] = pd.DataFrame(cor_test_data)
        print("파일을 불러왔습니다.")
else:
    with open("cor_test_data.txt", 'wb') as F:
        pickle.dump(correction_document, F, pickle.HIGHEST_PROTOCOL)
        print("파일을 생성하였습니다.")

파일을 생성하였습니다.
파일을 생성하였습니다.


In [11]:
# 결측값(NaN, Null) 확인
train_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [12]:
# 결측값(NaN, Null) 제거
train_data.dropna(inplace = True)
test_data.dropna(inplace = True)

In [13]:
train_data.shape

(150000, 3)

In [14]:
# 빈값 확인 
train_drop = [ i for i, v in enumerate(train_data['document']) if str(v).strip() == "" ]
test_drop = [ i for i, v in enumerate(test_data['document']) if str(v).strip() == "" ]

train_data.loc[train_data.index[train_drop]]

,id,document,label
404,4221289,,0
412,9509970,,1
470,10147571,,1
668,1600635,,0
972,7425748,,0
...,...,...,...
149445,7133917,,0
149630,3508604,,0
149718,7690797,,1
149773,9233162,,0


In [15]:
# 빈값 제거
train_data.drop(train_drop, inplace=True)
test_data.drop(test_drop, inplace=True)

# 인덱스 재정렬
train_data.reset_index(drop=True,  inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [18]:
train_data.shape

(148526, 3)

# 3. 데이터 코드화

(LSTM 분석을 위해서는 벡터화된 데이터셋이 필요함.)</br>
토큰화는 Kkma, Komoran 두가지로 테스트 예정</br>
둘 중 프로젝트에 더 적합한 모습을 보이는 라이브러리로 진행을 위한 테스트


In [19]:
import jpype

from konlpy.tag import Kkma, Hannanum, Komoran, Okt

Kkma = Kkma()
Komoran = Komoran()
Okt = Okt()
Hannanum = Hannanum()

for i in range(4):
    print("="*100)
    print(train_data['document'][i])
    print("Kkma", Kkma.morphs(train_data['document'][i]))
    print("Komran",Komoran.morphs(train_data['document'][i]))
    print("Okt", Okt.morphs(train_data['document'][i], norm=True))   # norm: 정규화 처리 / stem: 각 단어에서 어간을 추출
    print("Hannanum", Hannanum.morphs(train_data['document'][i]))

아 더빙 진짜 짜증 나네요 목소리
Kkma ['아', '아', '더빙', '진짜', '짜증', '나', '네요', '목소리']
Komran ['아', '더빙', '진짜', '짜증', '나', '네요', '목소리']
Okt ['아', '더빙', '진짜', '짜증', '나네요', '목소리']
Hannanum ['아', '더빙', '진짜', '짜증', '나', '이', '네', '요', '목소리']
흠포스터보고 초등학생영화 줄 오버 연기조차 가볍지 않구나
Kkma ['흠', '포스터', '보고', '초등학생', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나']
Komran ['흠', '포스터', '보고', '초등학생', '영화', '주', 'ㄹ', '오버', '연기', '조차', '가볍', '지', '않', '구나']
Okt ['흠', '포스터', '보고', '초등학생', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']
Hannanum ['흠포스터보', '이', '고', '초등학생영화', '주', 'ㄹ', '오버', '연기', '조차', '가볍', '지', '않', '구나']
너무재밓었다그래서보는것을추천한다
Kkma ['너무', '재', '밓', '어', '었', '다', '그래서', '보', '는', '것', '을', '추천', '하', 'ㄴ다']
Komran ['너무재밓었다그래서보는것을추천한다']
Okt ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다']
Hannanum ['너무재밓었다그래서보는것을추천한다']
교도소 이야기구먼 솔직히 재미는 없다 평점 조정
Kkma ['교도소', '이야기', '구', '멀', 'ㄴ', '솔직히', '재미', '는', '없', '다', '평점', '조정']
Komran ['교도소', '이야기', '이', '구먼', '솔직히', '재미', '는', '없', '다', '평점', '조정']
Okt ['교도소', '이야기',

_테스트 결과_<br>
맞춤법과 띄어쓰기가 보정된 문장을 단어로 분리하는 경우 전체적으로 비슷한 성능을 보임<br>


다만 속도에서 [독보적으로 빠른](https://mr-doosun.tistory.com/22) **Okt**를 사용하는 것이 적절하다고 판단 



## 문장 → 단어로 분할 / 불용어 사전 제작

In [20]:
import jpype

from konlpy.tag import Okt
from tqdm import tqdm

stopwords = ['의','가','이','은','들','는','걍','과', '의', '도','을', '것', '를','으로','자','에','와','한','하다',
             'ㄴ', '다','에서', '하는', '나', '자', '고', '이다','수','데','인데']

# 품사 태깅
tagged_train_data = []
train_error_sentence = []

for i in tqdm(range(train_data.shape[0])):
    try:
        temp_x = Okt.morphs(train_data['document'][i], norm=True)                 # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords]             # 불용어 제거
        tagged_train_data.append(temp_x)
    except:
        tagged_train_data.append([])
        train_error_sentence.append(i)

print("훈련데이터 에러 발생 수:", len(train_error_sentence))

100%|█████████████████████████████████████████████████████████████████████████| 148526/148526 [06:12<00:00, 398.51it/s]

훈련데이터 에러 발생 수: 0


In [21]:
stopwords = ['의','가','이','은','들','는','걍','과', '의', '도','을', '것', '를','으로','자','에','와','한','하다', 
             'ㄴ', '다','에서', '하는', '나', '자', '고', '이다','수','데','인데']   

# 품사 태깅
tagged_test_data = []
test_error_sentence = []

for i in tqdm(range(test_data.shape[0])):
    try:
        temp_x = Okt.morphs(test_data['document'][i], norm=True)                 # 토큰화
        temp_x = [word for word in temp_x if not word in stopwords]           # 불용어 제거
        tagged_test_data.append(temp_x)
    except:
        tagged_test_data.append([])
        test_error_sentence.append(i)

print("훈련데이터 에러 발생 수:", len(test_error_sentence))

100%|███████████████████████████████████████████████████████████████████████████| 49483/49483 [02:31<00:00, 327.43it/s]

훈련데이터 에러 발생 수: 0


In [22]:
import os.path
import pickle

# tagged_train_data 저장
file = 'tagged_train_data.txt'

if os.path.isfile(file):
    print('파일이 존재합니다. 불러옵니다')
    with open('tagged_train_data.txt', 'rb') as f:
        tagged_train_data = pickle.load(f)
        print("불러오기 완료.")
else:
    with open("tagged_train_data.txt", 'wb') as F:
        pickle.dump(tagged_train_data, F, pickle.HIGHEST_PROTOCOL)
        print("파일을 생성하였습니다.")
        
        
# tagged_test_data 저장
file = 'tagged_test_data.txt'

if os.path.isfile(file):
    print('파일이 존재합니다. 불러옵니다')
    with open('tagged_test_data.txt', 'rb') as f:
        tagged_test_data = pickle.load(f)
        print("불러오기 완료.")
else:
    with open("tagged_test_data.txt", 'wb') as F:
        pickle.dump(tagged_test_data, F, pickle.HIGHEST_PROTOCOL)
        print("파일을 생성하였습니다.")

파일을 생성하였습니다.
파일을 생성하였습니다.


# 기계가 텍스트를 숫자로 처리할 수 있도록 정수 인코딩을 진행
<br>
다만 태깅한 품사를 모두 사용해서 학습한다면, 1, 2회 사용된 <br>
즉 너무 적은 단위로 사용된 단어들까지 학습된다. <br>
하면 데이터 분석에 효율성이 떨어질 수 있다. (혹은 과적합)<br>
하여 자주 사용된 단어(3회 이상)만 추려서 인공신경망 학습에 사용될 수 있도록<br>
3회 이하 사용된 단어들은 제거 한다. 

#### 3회 이하로 사용된 단어들이 얼마나 있는지 확인한다.

In [ ]:
# 우선 정수 인코딩 진행
from keras.preprocessing.text import Tokenizer

tokenizer_maxlen = Tokenizer()
tokenizer_maxlen.fit_on_texts(tagged_train_data)

print(tokenizer_maxlen.word_index)  # word_index => {'하': 1, 'ㄴ': 2, '영화': 3, '다': 4, '고': 5, ... ✨자주 사용된 단어가 먼저 나열
                                    # word_counts => OrderedDict([('아', 154), ('더빙', 5), ('진짜', 69) ...
                                    # word_docs => defaultdict(<class 'int'>, {'목소리': 2, '짜증나': 8, ...


In [24]:
threshold = 4
total_cnt = len(tokenizer_maxlen.word_index)                # 모든 토큰의 수
rare_cnt = 0                                                # 3번 이하 등장 단어 개수
total_freq = 0                                              # 토근화된 단어의 총합
rare_freq = 0                                               # 토근화된 단어 중 3번 이하 사용된 단어의 총합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer_maxlen.word_counts.items():       # word_counts: 해당 단어의 빈도수를 반환
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if value < threshold:
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 2번 이하인 희귀 단어의 수:', rare_cnt)
print("전체 품사 집합에서 2회 이하 사용된 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

NameError: name 'tokenizer_maxlen' is not defined

In [ ]:
# 0번 패딩 토큰을 고려하여 + 1
voca_size = total_cnt - rare_cnt + 1
print(f'즉, 총 단어의 수는 {total_cnt}개 / 그 중 2회 이하 사용된 단어의 수는 {rare_cnt}')
print("2회 이하 사용된 단어를 제거한 == 남길 단어(Embedding)의 사이즈:", voca_size - 1)

### 임베딩 할 단어 voca_size 를 확인</n>

### *본 데이터 임베딩 진행



In [ ]:
tokenizer = Tokenizer(num_words=voca_size)                           # Tokenizer(vaca_size) => 토근화 진행하면서 max_embadding(100, 12, 4 ...)를 설정
# 학습 데이터 
tokenizer.fit_on_texts(tagged_train_data)                            # fit_on_texts => Updates internal vocabulary based on a list of texts.
enbadded_train_data = tokenizer.texts_to_sequences(tagged_train_data)    # texts_to_sequences => Transforms each text in texts to a sequence of integers.

# 검증 데이터 
tokenizer.fit_on_texts(tagged_test_data)                             # fit_on_texts => Updates internal vocabulary based on a list of texts.
embadded_test_data = tokenizer.texts_to_sequences(tagged_test_data)

len(enbadded_train_data)

In [ ]:
ex_train_data

In [ ]:
# 테스트를 위해 데이터 분리 및 임베딩 데이터 추가
ex_train_data = train_data
ex_test_data = test_data

ex_train_data.insert(ex_train_data.shape[1], "token", enbadded_train_data)
ex_test_data.insert(ex_train_data.shape[1], "token", embadded_test_data)

In [ ]:
ex_train_data.to_csv("ex_train_data.csv", encoding='utf-8-sig')

In [ ]:
# 빈 리스트 행 확인
train_drop = [i for i, v in enumerate(ex_train_data['token']) if v == []]
test_drop = [i for i, v in enumerate(ex_test_data['token']) if v == []]

print(len(train_drop))
print(len(test_drop))
print(train_drop)

In [ ]:
# 빈 리스트 행 제거 및 확인
ex_train_data.drop(train_drop, inplace=True)
ex_test_data.drop(test_drop, inplace=True)

ex_train_data.reset_index(drop=True, inplace=True)
ex_test_data.reset_index(drop=True, inplace=True)


In [ ]:
refine_test_data.shape 

# 모델 학습을 위한 sequence padding 진행


In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in refine_train_data["token"]))
print('리뷰의 평균 길이 :',sum(map(len, refine_train_data["token"]))/len(refine_train_data["token"]))

import matplotlib.pyplot as plt
plt.hist([len(s) for s in refine_train_data["token"]], bins=60)
plt.xlabel('length ')
plt.show()

In [ ]:
def belong(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 30
for max_len in range(30, 60, 5):
    belong(max_len, refine_data["token"])

## padding 길이 설정 후 => train_x 생성

In [ ]:
from keras.preprocessing import sequence

train_x = sequence.pad_sequences(refine_data['token'], maxlen=40)

print(train_x[:3], type(traind_x))

## refine_train_data에서 라벨값을 추려 => train_label 생성

In [ ]:
train_label = refine_data['label'].to_numpy()
print(train_label[:3], type(train_label))

## test_x 생성

In [ ]:
test_x = sequence.pad_sequences(refine_test_data['token'], maxlen=40)
print(test_x[:3], type(test_x))

## refine_test_data에서 라벨값을 추려 => test_label 생성

In [ ]:
test_label = refine_test_data['label'].to_numpy()
print(test_label[:3], type(test_label))

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(voca_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(train_x, train_label, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(test_x, test_label)[1]))